In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
import numpy as np
import scipy.stats
from IPython.display import YouTubeVideo


# Simulación y modelos generativos 


## ¿Qué es y para que sirve simular?


### Definiciones de simular

- ["Hacer que algo parezca real no siéndolo"](https://www.rae.es/dpd/simular)
- Del latín [simulâre: "copiar, representar"](https://es.wiktionary.org/wiki/simulaci%C3%B3n) o [simulatio "acción y efecto de imitar algo"](http://etimologias.dechile.net/?simulacio.n)
- Reproducir artificialmente un fenómeno o las relaciones entrada-salida de un sistema


Una simulación digital es la aplicación de un modelo computacional para la predicción de eventos físicos o el comportamiento de sistemas de ingeniería

<img src="images/paradigms.jpg" width="600">

### ¿Por qué simulamos en ingeniería y ciencia?

La simulación nos permite explorar nuevas teorías o diseñar nuevos experimentos para probar dichas teorias

Usando simulación podemos estudiar fenomenos que son de plano no observables o muy difíciles/caros/peligrosos/imprácticos de observar o medir

Usando simulaciones podemos 

- analizar un sistema antes de haberlo construido
- analizar situaciones a las que el sistema aun no ha sido expuesto
- realizar predicciones sobre el comportamiento del sistema
- analizar la influencia e inter-relaciones entre las variables del sistema y en consecuencia entender mejor su operación. [Ceteris paribus](https://en.wikipedia.org/wiki/Ceteris_paribus)

Los simuladores son también ampliamente usados en educación

Con el rápido avance y la disminución en costos de la computación, la simulación digital se ha vuelto una herramienta clave en muchas disciplinas (eléctrica, mecánica, química, aeroespacial, nuclear, biomédica, materiales, etc)

> [Los principales desafíos de los simuladores están en la validación, verificación y medición de incerteza de los modelos](http://www.mcs-uab.com/docs/NSF.Simluation-Based_Engineering_Science.2006.pdf)



<img src="images/morpho.jpg" width="700">

## ¿Qué son los modelo generativos?


:::{epigraph}

[What I cannot create, I do not understand](https://jcs.biologists.org/content/joces/130/18/2941.full.pdf)

-- Richard Feynman

:::



Sea un observación descrita por $x$ que viene de una distribución de probabilidad $p^*(x)$. Un modelo generativo es una aproximación 

$$
p_\theta(x) \approx p^*(x)
$$ 

con la cual podemos generar nuevos ejemplos aleatorios de $x$. Sea por ejemplo la siguiente muestra de diez datos escalares y continuos

In [ ]:
with open("../data/mistery_data.npy", "rb") as f:
    data = np.load(f)
print(data)

Como se vio en la unidad anterior podemos ajustar un modelo probabilístico usando estimación de máxima verosimilitud

En este ejemplo seleccionamos una distribución normal y la ajustamos con `scipy.stats`

In [ ]:
# Seleccionamos una distribución
dist = scipy.stats.norm 

# Ajustamos los parámetros con MLE
args = dist.fit(data) 

# Estos son los parámetros ajustados
print(args)

Luego creamos un modelo con los parámetros ajustados. Al ser un modelo probabilístico podemos generar nuevos datos

In [ ]:
# Crear modelo
model = dist(*args) 

# Generar datos
synthetic_data = model.rvs(size=1000) 
print(synthetic_data[:10])

**Secreto relevado:** Los datos fueron creados a partir de una distribución normal con media 10 y desviación estándar 4

A pesar de tener pocas observaciones el modelo parece haberse ajustado bien, pero ¿Qué hubiera pasado si hubieramos escogido otra distribución para ajustar? ¿Cómo evaluamos si el modelo escogido es adecuado?

Pasemos ahora a un ejemplo más complejo, cuyos datos no se distribuyen normal. Ajustemos una distribución normal multivariada y generemos algunos datos ¿Le parecen aceptables?

In [ ]:
with open("../data/mnist_data.npy", "rb") as f:
    data = np.load(f) 
    
# TODO: USAR EJEMPLOS DE UN SOLO DIGITO
    
dist = scipy.stats.multivariate_normal 
mu = np.mean(data.reshape(-1, 28*28), axis=0) # MLE de la media
cov = np.cov(data.reshape(-1, 28*28), rowvar=False) # MLE de la covarianza
model = dist(mean=mu, cov=cov+0.001*np.eye(28*28)) # Crear modelo
new_data = model.rvs(size=10) # Generar datos

In [ ]:
real_list = [hv.Image(sample).opts(width=100, height=100) for sample in data]
synt_list = [hv.Image(sample.reshape(28, 28)) for sample in new_data]

layout = hv.Layout(real_list + synt_list).cols(10)
layout.opts(hv.opts.Image(width=80, height=80, cmap='binary', xaxis=None, yaxis=None))

Lamentablemente los ejemplos de la vida real son demasiado complejos para modelarse con distribuciones sencillas

Es mucho más conveniente asumir que existe una variable oculta o latente $z$ con una distribución sencilla $p(z)$ que luego se modifica a través de una transformación  $p(x|z)$ para obtener un ejemplo de $x$

En ese caso buscamos modelar la evidencia o verosimilitud marginal

$$
p(x) = \int_{\mathcal{z\in Z}} p(x|z) p(z) \,dz = \int_{\mathcal{z\in Z}} p(x, z) \,dz
$$

Esto además nos permite condicionar el sistema generador con variables que nos interesen

> [Los modelos generativos se investigan muy activamente hoy en día](https://openai.com/blog/generative-models)

[Ejemplo](https://arxiv.org/abs/1807.03039) de un modelo generativo con variable latente entrenado sobre imágenes de rostros humanos.

In [ ]:
YouTubeVideo('exJZOC3ZceA')

## Recordatorio del Teorema de Bayes

De las propiedades de las probabilidades condicionales y la ley de probabilidades totales podemos escribir

$$
p(y|x) = \frac{p(x|y) p(y)}{p(x)} = \frac{p(x|y) p(y)}{\int p(x|y) p(y) \,dy}
$$

Tipicamente $x$ representa un conjunto de datos que hemos observado a través de un experimento e $y$ algún parámetro que queremos estimar. 

**Reflexione:** ¿Qué representan $p(y)$ y $p(y|x)$? ¿Cuándo es conveniente usar el teorema de bayes?


**Desafio:** La marginalización de la variable latente, el cálculo de la esperanza de una variable continua o el cálculo de la evidencia en el teorema de Bayes requiere resolver integrales que pueden ser muy complicadas

